In [1]:
import sys
# set python syspath to point out location of our self-writing module
sys.path.append("/home/ponshane/work_dir/CLTM/src/codebase/")

import configparser
from datetime import datetime
from pymongo import MongoClient
from Sentence_Segmentation import Sentence_Segmentation
from Chinese_Tokenizer import Tokenizer
from Chinese_POSTagger import POSTagger
import re

Can't not use customized dictionary


In [2]:
### init and read config
config = configparser.ConfigParser()
config.read('../config.ini')

MongoDB = config["ADM"]["Database"]
MongoUser = config["ADM"]["User"]
MongoPW = config["ADM"]["PW"]

###連接MONGO
uri = "mongodb://" + MongoUser + ":" + MongoPW + "@140.117.69.70:30241/" +\
MongoDB + "?authMechanism=SCRAM-SHA-1"

client = MongoClient(uri)
db = client.ComparableWiki

In [ ]:
###
### test area
###

one_document = db.ENZH.find_one()
#print(one_document)

chi_results = Sentence_Segmentation(one_document["ZH-Content"], keep_digits=False)
print(chi_results)

chi_tokens = Tokenizer(chi_results)
print(chi_tokens)

chi_pos = POSTagger(chi_tokens)
print(chi_pos)

In [4]:
# 適用於中文 NLP Process 的 Update，因為中英文產出欄位不同
def update(target_collection, doc_id, sentences, tokens, pos):
    Result = {"sentences":sentences, "tokens": tokens, "pos": pos}
    target_collection.update_one({"_id": doc_id},
                      {
                          "$set":{
                              "chi_result": Result,
                              "chi_nlp_process": True
                          }
                      })

In [5]:
target_collection = db.ENZH
num = target_collection.count({"chi_nlp_process": {"$exists": False}})
print("{0} documents need to be processed.".format(num))

389389 documents need to be processed.


In [ ]:
docs = target_collection.find({"chi_nlp_process":{"$exists": False}},{"_id":1, "ZH-Content":1}, no_cursor_timeout=True)

# improve version
start_time = datetime.now()

error_list = list()
index = 0

for each_document in docs:
    sentences = Sentence_Segmentation(each_document["ZH-Content"], keep_digits=False)
    tokens = Tokenizer(sentences)
    pos = POSTagger(tokens)
    
    try:
        update(target_collection, each_document["_id"], sentences, tokens, pos)
    except:
        error_list.append(each_document["_id"])
    
    index += 1
    if(index % 2500 ==0):
        print("Already process %d documents" % index)
        time_elapsed = datetime.now() - start_time
        print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

print("{0} documents got some problems".format(len(error_list)))
print(error_list)

docs.close()
# Time elapsed (hh:mm:ss.ms) 6:14:58.814529